In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, lit
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
import sys
sys.path.append('../')
from scripts.scrape_oldlistings import scrape_postcodes, scrape_postcodes_from_file, get_oldlisting_data, get_remaining_suburbs, get_remaining_oldlisting_data, convert_csv_to_parquet


In [2]:
scrape_postcodes_from_file()

,suburb,postcode
0,Abbotsford,3067
1,Aberfeldie,3040
2,Aberfeldy,3825
3,Acheron,3714
4,Addington,3352
...,...,...
1698,Yeodene,3249
1699,Yinnar,3869
1700,Youanmite,3646
1701,Yundool,3727


In [3]:
get_oldlisting_data()

Getting data for Abbotsford, 3067. Suburb 0 of 1703
num pages: 96
Getting data for Aberfeldie, 3040. Suburb 1 of 1703
num pages: 11
Getting data for Aberfeldy, 3825. Suburb 2 of 1703
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Aberfeldy/3825/rent/. Continuing with other URLs.
Getting data for Acheron, 3714. Suburb 3 of 1703
num pages: 1
Getting data for Addington, 3352. Suburb 4 of 1703
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Addington/3352/rent/. Continuing with other URLs.
Getting data for Agnes, 3962. Suburb 5 of 1703
num pages: 1
Getting data for Aireys+Inlet, 3231. Suburb 6 of 1703
num pages: 5
Getting data for Airport+West, 3042. Suburb 7 of 1703
num pages: 45
Getting data for Albanvale, 3021. Suburb 8 of 1703
num pages: 12
Getting data for Albert+Park, 3206. Suburb 9 of 1703
num pages: 47
Getting data for Alberton, 3971. Suburb 10 of 1703
num pages: 1
Getting data for Albion, 3020. Suburb 11 of 1703
num pages: 25
Getting data f

In [2]:
get_remaining_suburbs()

In [9]:
get_remaining_oldlisting_data()

Getting data for West+Melbourne, 3003. Suburb 0 of 97
num pages: 106
Getting data for Westmeadows, 3049. Suburb 1 of 97
num pages: 22
Getting data for Westmere, 3351. Suburb 2 of 97
num pages: 1
Getting data for Wheelers+Hill, 3150. Suburb 3 of 97
num pages: 48
Getting data for White+Hills, 3550. Suburb 4 of 97
num pages: 15
Getting data for Whitfield, 3733. Suburb 5 of 97
num pages: 1
Getting data for Whitlands, 3733. Suburb 6 of 97
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Whitlands/3733/rent/. Continuing with other URLs.
Getting data for Whittington, 3219. Suburb 7 of 97
num pages: 14
Getting data for Whittlesea, 3757. Suburb 8 of 97
num pages: 14
Getting data for Whorouly, 3735. Suburb 9 of 97
num pages: 2
Getting data for Wickliffe, 3379. Suburb 10 of 97
Error 404: Not found https://www.oldlistings.com.au/real-estate/VIC/Wickliffe/3379/rent/. Continuing with other URLs.
Getting data for Wilby, 3728. Suburb 11 of 97
num pages: 1
Getting data for Willaura, 

In [3]:
convert_csv_to_parquet()

In [2]:
spark = (
    SparkSession.builder.appName("Preliminary Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/08 21:54:47 WARN Utils: Your hostname, LAPTOP-UFNFC5IK resolves to a loopback address: 127.0.1.1; using 172.17.209.63 instead (on interface eth0)
24/09/08 21:54:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/08 21:54:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
sdf = spark.read.parquet("../data/landing/oldlisting/oldlisting.parquet")
sdf.show()

+----------+--------+--------------------+--------------+--------------+----+-----+----+----------+--------------------+--------------------+
|    suburb|postcode|             address|      latitude|     longitude|beds|baths|cars|house_type|               dates|           price_str|
+----------+--------+--------------------+--------------+--------------+----+-----+----+----------+--------------------+--------------------+
|Abbotsford|    3067|203/1 TURNER ST, ...|-37.7993152002|144.9975063002| 2.0|  2.0| 1.0| Unit/apmt|  ['September 2024']|   ['$700 per week']|
|Abbotsford|    3067|117 TURNER STREET...|    -37.799524|   144.9997368| 3.0|  2.0| 1.0|     House|  ['September 2024']| ['$1,450 per week']|
|Abbotsford|    3067|4/85 NICHOLSON ST...|   -37.8062248|   144.9966012| 3.0|  2.0| 1.0|     House|['September 2024'...|['$800 per week',...|
|Abbotsford|    3067|3/82 TRENERRY CRE...|    -37.799923|    144.999835| 2.0|  2.0| 1.0|      Unit|['September 2024'...|['$840', '$900 p....|
|Abbot

In [13]:
# Create a DataFrame where each entry includes its URL
flattened_df = df.selectExpr("stack(" + str(len(df.columns)) + "," + ",".join([f"'{c}', `{c}`" for c in df.columns]) + ") as (id, properties)")

# Expand the properties into separate columns
expanded_df = flattened_df.select(
    "id",
    "properties.*"  # Expands the nested columns directly
)

AnalysisException: [DATATYPE_MISMATCH.STACK_COLUMN_DIFF_TYPES] Cannot resolve "stack(39, https://www.domain.com.au/1-36-clarendon-st-east-melbourne-vic-3002-16665538, https://www.domain.com.au/1-36-clarendon-st-east-melbourne-vic-3002-16665538, https://www.domain.com.au/1-tarver-st-port-melbourne-vic-3207-14253235, https://www.domain.com.au/1-tarver-st-port-melbourne-vic-3207-14253235, https://www.domain.com.au/101-beaconsfield-parade-albert-park-vic-3206-17000597, https://www.domain.com.au/101-beaconsfield-parade-albert-park-vic-3206-17000597, https://www.domain.com.au/11-peacock-street-brighton-vic-3186-17057578, https://www.domain.com.au/11-peacock-street-brighton-vic-3186-17057578, https://www.domain.com.au/14a-ethel-street-malvern-vic-3144-16398071, https://www.domain.com.au/14a-ethel-street-malvern-vic-3144-16398071, https://www.domain.com.au/15a-elizabeth-street-brighton-vic-3186-17183719, https://www.domain.com.au/15a-elizabeth-street-brighton-vic-3186-17183719, https://www.dom

In [8]:
# Extract all column names (URLs)
column_names = df.columns

schema = StructType([
    StructField("id", StringType(), True),
    StructField("url", StringType(), True),
    StructField("name", StringType(), True),
    StructField("cost_text", StringType(), True),
    StructField("rooms", ArrayType(StringType()), True),
    StructField("parking", ArrayType(StringType()), True),
    StructField("desc", StringType(), True),
    StructField("property_type", StringType(), True),
    StructField("date_available", StringType(), True),
    StructField("bond", StringType(), True),
    StructField("property_features", ArrayType(StringType()), True),
    StructField("coordinates", ArrayType(StringType()), True)
])

empty_df = spark.createDataFrame([], schema)

# Iterate over each column and add its data to the DataFrame
for column in column_names:
    new_rows = df.select(
        lit(column).alias("id"),
        col(f"{column}.url").alias("URL"),
        col(f"{column}.name").alias("name"),
        col(f"{column}.cost_text").alias("cost_text"),
        col(f"{column}.rooms").alias("rooms"),
        col(f"{column}.parking").alias("parking"),
        col(f"{column}.desc").alias("desc"),
        col(f"{column}.property_type").alias("property_type"),
        col(f"{column}.date_available").alias("date_available"),
        col(f"{column}.bond").alias("bond"),
        col(f"{column}.property_features").alias("property_features"),
        col(f"{column}.coordinates").alias("coordinates")
    )
    empty_df = empty_df.union(new_rows)

# Show the transformed DataFrame
empty_df.show(truncate=False)

+------------------------------------------------------+--------------------------------------------------------------------------------+----------------------------------------------+--------------------------------------------------+-----------------+-----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------+--------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------